In [4]:
import random
import numpy as np
import copy
import random
import itertools
import re
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cmath

from scipy.optimize import curve_fit
import sys
import multiprocessing as mp

from tqdm import tqdm
benchmark = 2./3.

In [5]:
def create_init_state(max_qubits):

    all_init_states, all_init_amps = [],[]
    q_range = range(2,max_qubits+1)

    for n_q in q_range:

        #defining initial state
        init_state = []
        for i in range(2**n_q):
            x = [np.binary_repr(i, width=n_q)]
            res = [[int(i) for i in sub] for sub in x]
            init_state.extend(res)

        all_init_states.append(init_state)

        #generating random amplitudes
        init_amps = []
        init_amps = np.abs(np.random.uniform(0,1,2**n_q))
        # for i in range(2**n_q):
        #     x = cmath.rect(random.uniform(0.,1.),0.)
        #     init_amps.append(x)
        init_amps = np.sqrt(np.array(init_amps)/np.sum(np.array(init_amps))) + 0*1j

        all_init_amps.append(init_amps)

    return all_init_amps, all_init_states


In [6]:
def load_random_cnots_dynamic(file):
        with open(file, 'r') as f:
            data = f.read()
        list_of_initial_cnots = [[int(cnot.split()[0]),int(cnot.split()[1])] for cnot in re.findall('\d{1,2} \d{1,2}',data)]
        return list_of_initial_cnots

def compute_t_layer_displacement_and_cnots(gates):
    displacement = 0
    displacement_list = []
    cnots = 0
    selected = [False]*len(gates)
    for idx1,gate1 in enumerate(gates):
        displacement = 0
        cnots = 0
        if selected[idx1]==True:
            continue
        cnots += 3
        displacement += (1+abs(gate1[1]-gate1[0]))
        cnots += 1
        displacement_list.append([gate1,displacement,cnots])
        #print(gate1,displacement,cnots)
        prev = gate1[1]
        selected[idx1]=True
        for idx2,gate2 in enumerate(gates[idx1+1:]):
            if selected[idx1+idx2+1]==True:
                break
            if gate2[0]==gate1[0]:
                selected[idx1+idx2+1]=True
                cnots += 1
                displacement += abs(gate2[1]-prev)
                displacement_list.append([gate2,displacement,cnots])
                #print(gate2,displacement,cnots)
                prev = gate2[1]
            else:
                break
        displacement+= (1+abs(gate1[0]-prev))
        cnots += 3
        displacement_list.append([displacement,cnots])
        #print(displacement,cnots)
    return displacement_list

In [7]:
def load_random_cnots_static(file):
        with open(file, 'r') as f:
            data = f.read()
        pattern = r'\[\d+(?:,\s*\d+)*\]'
        list_of_initial_cnots = [[int(cnot.split(",")[0][1]),int(cnot.split(",")[1][1])] for cnot in re.findall(pattern,data)]
        return list_of_initial_cnots

def load_random_cnots_dynamic(file):
        with open(file, 'r') as f:
            data = f.read()
        list_of_initial_cnots = [[int(cnot.split()[0]),int(cnot.split()[1])] for cnot in re.findall('\d{1,2} \d{1,2}',data)]
        return list_of_initial_cnots

In [8]:
def ordering(in_state,fi_state,o_amps):

    i_state = copy.deepcopy(np.array(in_state))
    f_state = copy.deepcopy(np.array(fi_state))
    new_amps = [0]*len(i_state)

    for i in range(len(i_state)):
        x = np.where((f_state == i_state[i]).all(axis=1))
        #new_amps[i] = np.sqrt(np.sum(np.array(o_amps)[x]**2))
        new_amps[i] = np.abs(np.sum(np.array(o_amps)[x]))

    new_amps = np.array(new_amps)
    return new_amps

In [9]:
def CNOT(cc,tt):
    if cc == 1:
        if tt == 1:
            tt = 0
        else:
            tt = 1

    return cc,tt

def generate_transformation_matrix(n_q,CNOT_seq):

        tmat_i = np.identity(n_q)
        tmat_f = np.identity(n_q)
        for i in range(0,len(CNOT_seq)):
            #print(self.CNOT_seq)
            c = int(CNOT_seq[i][0])
            t = int(CNOT_seq[i][1])

            for j in range(0,n_q):
                tmat_i[t][j] = int(tmat_i[c][j]) ^ int(tmat_i[t][j])

            tmat_f = tmat_i
            #self.tmat_i = np.identity(self.n_q)
        return tmat_f

def compile_circuit(num_qubits, trans_mat):
        A = copy.deepcopy(trans_mat)
        A = A.astype(np.bool_)
        gates = []
        for col in range(len(A)):
            dia_one = 1
            if A[col,col] == 0:
                dia_one = 0
            for row in range(col+1,len(A)):
                if A[row,col]==1:
                    if dia_one==0:
                        A[col] = A[col]^A[row]
                        gates.append([row,col])
                        dia_one=1
                    A[row] = A[row]^A[col]
                    gates.append([col,row])

        for col in range(len(A)-1,-1,-1):
            for row in range(col-1,-1,-1):
                if A[row,col]==1:
                    A[row] = A[row]^A[col]
                    gates.append([col,row])
        assert((A==np.identity(num_qubits,dtype=np.ubyte)).all())
        return gates

def circuit_classic(ncnots_c,cnots_c,nstates_c,states_c,amps_c):

    #prob = np.sqrt((1 - prob)**2 + (prob)**2)
    nstates = range(nstates_c)
    for cnot in cnots_c:
        control,target = int(cnot[0]),int(cnot[1])
        for id_state in nstates:
            state = states_c[id_state]
            if state[control] == 1:
                new_val = (int(state[target])+1)%2
                state[target] = new_val

    amps_c = np.array(amps_c)

    return states_c, amps_c

def circuit_help(ncnots_c,cnots_c,nstates_c,states_c,amps_c,prob):

    assert np.sum(np.abs(amps_c)**2) <= 1+1e-6 and np.sum(np.abs(amps_c)**2) >= 1-1e-6

    phase = np.sqrt(1 - prob**2)*1j#np.sqrt(2*(1-prob))*1j#np.arctan((1.-prob)/(1.+prob))

    #if prob == 1: phase = 0
    #prob = 1. - (1-prob)/2 + (1-prob)/2*1j#np.sqrt((1 + prob**2)/2.)
    nstates = range(nstates_c)
    amps_c = np.array([complex(x) for x in amps_c])
    amp_e = copy.deepcopy(amps_c)
    states_c  = np.array(states_c)

    for cnot in cnots_c:

        control,target = int(cnot[0]),int(cnot[1])
        control_args = np.where(states_c[:,control] == 1)
        target_args0 = np.where(states_c[:,target] == 0)
        target_args1 = np.where(states_c[:,target] == 1)

        ta0 = [int(x) for x in np.intersect1d(control_args, target_args0)]
        ta1 = [int(x) for x in np.intersect1d(control_args, target_args1)]

        amp_e[ta0] = prob*amps_c[ta1] + phase*amps_c[ta0]
        amp_e[ta1] = prob*amps_c[ta0] + phase*amps_c[ta1]

        amps_c = copy.deepcopy(amp_e)

    #print(np.sum(np.abs(amp_e)**2))
    assert np.sum(np.abs(amp_e)**2) <= 1+1e-6 and np.sum(np.abs(amp_e)**2) >= 1-1e-6

    return states_c, amps_c, amp_e


In [ ]:
ncnots_c = 1
cc = np.array([[0,1]])
init_amps, init_states = create_init_state(10)
nstates_c = 2**2
init_state = init_states[0]
init_amp = init_amps[0]
a,b,c,d = init_amp
print(init_amp)
print(a,b,c,d)
prob = np.cos((0.01)/2)
prob1 = np.sqrt(1-prob**2)
print(prob,1-(0.01)**2/4)
#print((2*prob**2-1)*c + 2j*prob*(np.sqrt(1-prob**2)*d))

#print('d,c', (2*prob**2-1)*d + 2j*prob*(np.sqrt(1-prob**2)*c))
#print(c*prob**2 + b*1j*prob1 + d*1j*prob*prob1)
print(b*prob**2 + c*(-prob1**2 + 1j*prob**2*prob1 ) + d*(-prob1**2*prob + 1j*prob*prob1))
print('\ninit amps',np.abs(init_amp))
c =circuit_help(ncnots_c,cc,nstates_c,init_state,init_amp,prob)
d = circuit_help(ncnots_c,cc,nstates_c,init_state,init_amp,1.)

print('\nclassical:\n', d[2],'\nactual\n',c[2])

In [ ]:
def operation1(case, n_qubit, init_states, init_amps, id, cnot_prob, atom_loss=1.):

    n_cnots = n_qubit//2

    path = f"D:\\IITBombay\\useqip\\URA\\DIG\\quantum_volume"

    if case == 'c':
        #CLASSICAL
        # classic_file = path + '/dynamic_compiled_' + str(n_qubit) + '/circuit_' + str(id)  + '_dynamic.txt'
        # classic_circ = np.array(load_random_cnots_dynamic(classic_file))
        classic_file = path + '/cnots_' + str(n_qubit) + '/circuit_' + str(id)  + '.txt'
        classic_circ = np.array(load_random_cnots_dynamic(classic_file))
        #classic_circ = np.array([[0,1],[0,1]])
        #classical_states, classical_amps = circuit(len(classic_circ), classic_circ, len(init_states), init_states, init_amps, False, 1)
        classical_states, dum ,classical_amps = circuit_help(len(classic_circ), classic_circ, len(init_states), init_states, init_amps, 1.)

        #classical_amps = np.array(ordering(init_states, classical_states, classical_amps))
        return len(classic_circ), classical_amps#, classical_states

    elif case == 's':
        #STATIC
        static_file = path + '/static_compiled_' + str(n_qubit) + '/circuit_' + str(id) + '_static.txt'
        static_compiled = np.array(load_random_cnots_static(static_file))-1.
        #static_states, static_amps = circuit(len(static_compiled),static_compiled,len(init_states),init_states,init_amps, True, cnot_prob)
        static_states, dum, static_amps = circuit_help(len(static_compiled),static_compiled,len(init_states),init_states,init_amps, cnot_prob)
        #static_amps = np.array(ordering(init_states, static_states, static_amps))
        return len(static_compiled), static_amps#, static_states

    elif case=='du':
        #DYNAMIC UNCOMPILED
        dynamic_uncompiled_file = path + '/cnots_' + str(n_qubit) + '/circuit_' + str(id)  + '.txt'
        dynamic_notcompiled = np.array(load_random_cnots_dynamic(dynamic_uncompiled_file))

        #dynamic_notcompiled_states, dynamic_notcompiled_amps =  circuit(len(dynamic_notcompiled), dynamic_notcompiled, len(init_states), init_states, init_amps, True, cnot_prob)
        dynamic_notcompiled_states, dum, dynamic_notcompiled_amps =  circuit_help(len(dynamic_notcompiled), dynamic_notcompiled, len(init_states), init_states, init_amps, cnot_prob)
        #dynamic_notcompiled_amps = np.array(ordering(init_states, dynamic_notcompiled_states, dynamic_notcompiled_amps))
        if atom_loss==1:
            #WITHOUT ATOM LOSS
            return len(dynamic_notcompiled), dynamic_notcompiled_amps#, dynamic_notcompiled_states,
        else:
            #WITH ATOM LOSS
            disp_notcompiled = compute_t_layer_displacement_and_cnots(dynamic_notcompiled)
            total_disps_notcompiled = 0.
            for i in disp_notcompiled:
                if len(i)==2: total_disps_notcompiled += i[0]
            dynamic_notcompiled_disp_amps = np.array(dynamic_notcompiled_amps)*atom_loss**total_disps_notcompiled
            return len(dynamic_notcompiled), dynamic_notcompiled_disp_amps#, dynamic_notcompiled_states

    elif case=='dc':

        #DYNAMIC COMPLIED WITHOUT ATOM LOSS
        dynamic_compiled_file = path + '/dynamic_compiled_' + str(n_qubit) + '/circuit_' + str(id)  + '_dynamic.txt'
        dynamic_compiled = np.array(load_random_cnots_dynamic(dynamic_compiled_file))[::-1]
        #print(dynamic_compiled)
        #dynamic_compiled = np.array([[0,1],[0,1]])
        #print(dynamic_compiled)

        #dynamic_compiled_states, dynamic_compiled_amps = circuit(len(dynamic_compiled), dynamic_compiled, len(init_states), init_states, init_amps, True, cnot_prob)
        dynamic_compiled_states, dum, dynamic_compiled_amps = circuit_help(len(dynamic_compiled), dynamic_compiled, len(init_states), init_states, init_amps, cnot_prob)
        #dynamic_compiled_amps = np.array(ordering(init_states, dynamic_compiled_states, dynamic_compiled_amps))
        if atom_loss==1:
            #WITHOUT ATOM LOSS
            return len(dynamic_compiled), dynamic_compiled_amps#, dynamic_compiled_states
        else:
            #WITH ATOM LOSS
            disp_compiled = compute_t_layer_displacement_and_cnots(dynamic_compiled)
            total_disps_compiled = 0.
            for i in disp_compiled:
                if len(i)==2: total_disps_compiled += i[0]
            dynamic_compiled_disp_amps = np.array(dynamic_compiled_amps)*atom_loss**total_disps_compiled
            return len(dynamic_compiled), dynamic_compiled_disp_amps#, dynamic_compiled_states

    else:
        raise Exception("Only acceptible cases are \n classical: c \n static: s \n dynamic uncompiled: ud \n dynamic compiled: dc")

In [ ]:
def hop_calc(case, max_qubits, init_amps, init_states, cnot_prob, atom_loss, n_trials):

    cnots_c, cnots, all_qubits_hps = [0]*(max_qubits-1),[0]*(max_qubits-1),[0]*(max_qubits-1)
    qubit_range = range(2,max_qubits+1)
    ncnots_c, ncnots = [0]*n_trials,[0]*n_trials

    for n_qubits in qubit_range:

        #initializing states, amps and hps

        init_state = init_states[n_qubits-2]
        init_amp = init_amps[n_qubits-2]

        #all_states_c, all_states = np.empty([n_trials, 2**n_qubits, n_qubits]), np.empty([n_trials, 2**n_qubits, n_qubits])
        #all_amps_c, all_amps = np.empty([n_trials,2**n_qubits]), np.empty([n_trials,2**n_qubits])
        all_hp = np.empty([n_trials])
        #paper_all_qv = np.empty([n_trials])

        #states_c, amps_c, states, amps = init_state, init_amp, init_state, init_amp

        for i in range(n_trials):

            ncnots_c[i], famps_c = operation1('c', n_qubits, init_state, init_amp, i, cnot_prob)
            ncnots[i], famps = operation1(case, n_qubits, init_state, init_amp, i, cnot_prob, atom_loss)

            probs_ascending_order_c = np.argsort(famps_c**2)
            probs_ascending_order = np.argsort(famps**2)
            sorted_probs_c = np.array(famps_c**2)[probs_ascending_order_c]
            sorted_probs = np.array(famps**2)[probs_ascending_order]

            median_classical = np.median(sorted_probs_c)

            hp = np.sum(sorted_probs[sorted_probs > median_classical])
            #paper_qv = len(sorted_probs[sorted_probs > median_classical])
            #paper_all_qv[i] = paper_qv
            all_hp[i] = hp
            #all_states_c[i], all_amps_c[i], all_states[i], all_amps[i] = np.array(states_c), np.array(amps_c), np.array(states), np.array(amps)
            #all_states_c[i], all_amps_c[i], all_states[i], all_amps[i] = np.array(states_c), np.array(amps_c), np.array(states), np.array(amps)

        cnots_c[n_qubits-2] = np.mean(ncnots_c)
        cnots[n_qubits-2] = np.mean(ncnots)

        case_hp = np.mean(all_hp)
        #paper_case_qv = (np.mean(paper_all_qv) - 2*np.sqrt(np.mean(paper_all_qv)*(1-np.mean(paper_all_qv)/n_trials)))/n_trials

        QV = np.max(qubit_range)
        if case_hp < benchmark:
            QV = n_qubits-1
            break

        all_qubits_hps[n_qubits-2] = case_hp

    return QV, cnots, all_qubits_hps


In [ ]:
#TO FIND MAXIMUM ERROR
def isHeavy(case, n_qubits, i_amps, i_states, cnot_prob, atom_loss, n_trials):

    ncnots_c, ncnots = [0]*n_trials,[0]*n_trials
    QV = True

    all_hp = np.empty([n_trials])
    #print(cnot_prob,'\n')
    a,b,c,d = i_amps[0:4]
    #print((2*cnot_prob**2-1)*c + 2j*cnot_prob*(np.sqrt(1-cnot_prob**2)*d))

    #print('d,c', (2*cnot_prob**2-1)*d + 2j*cnot_prob*(np.sqrt(1-cnot_prob**2)*c))
    for i in range(n_trials):

        #print('initial median: ', np.median(np.sort(np.abs(i_amps)**2)))
        ncnots_c[i], famps_c = operation1('c', n_qubits, i_states, i_amps, i, cnot_prob)
        ncnots[i], famps = operation1(case, n_qubits, i_states, i_amps, i, cnot_prob, atom_loss)
        #print('evolved probs:')
        #for k in range(len(i_states)): print( i_states[k],': ', np.array(famps_c)[k], np.array(famps)[k])
        #print('\n')


        probs_ascending_order_c = np.argsort(np.abs(famps_c)**2)
        probs_ascending_order = np.argsort(np.abs(famps)**2)
        sorted_probs_c = np.array(np.abs(famps_c)**2)[probs_ascending_order_c]
        sorted_probs = np.array(np.abs(famps)**2)[probs_ascending_order]
        median_classical = np.median(sorted_probs_c)

        sorted_states = copy.deepcopy(i_states)
        #print('sorted:')
        #for k in range(len(i_states)):print(np.array(i_states)[probs_ascending_order_c][k],': ', sorted_probs_c[k], np.array(sorted_states)[probs_ascending_order][k],': ',sorted_probs[k])
        #print([probs_ascending_order_c[len(probs_ascending_order_c)//2:]], np.array(np.abs(famps)**2)[probs_ascending_order_c[len(probs_ascending_order_c)//2:]])
        hp = np.sum(np.array(np.abs(famps)**2)[probs_ascending_order_c[len(probs_ascending_order_c)//2:]])
        hp_c = np.sum(np.array(np.abs(famps_c)**2)[probs_ascending_order_c[len(probs_ascending_order_c)//2:]])
        #print('median: ',median_classical,'hp_c: ', hp_c,'hp: ',hp,'\n')

        all_hp[i] = hp

    cnots_c = np.mean(ncnots_c)
    cnots = np.mean(ncnots)

    case_hp = np.mean(all_hp)
        #paper_case_qv = (np.mean(paper_all_qv) - 2*np.sqrt(np.mean(paper_all_qv)*(1-np.mean(paper_all_qv)/n_trials)))/n_trials

    if case_hp < benchmark:
        QV = False

    all_qubits_hps = case_hp

    return QV, cnots, all_qubits_hps


n_qubit = 7

n_trials = 999#min(999,2**max_qubits)
benchmark = 2./3.
atom_loss = 1.
# err_prob = x = np.arange(0.15,0.8,0.01)
# err_prob = np.cos(err_prob)
err_prob = np.arange(0.999,0.99999,0.00001)
err_prob = np.sqrt(4 - 5*(1-err_prob)) - 1
print(err_prob)
trials = 50
all_mer = [0]*trials
init_amps, init_states = create_init_state(n_qubit)
istate = init_states[n_qubit - 2]
iamp = init_amps[n_qubit - 2]
# iamp[0:4] = np.array([0.61969722+0.j, 0.64875737+0.j, 0.42085329+0.j, 0.13405875+0.j])
# iamp[4:] = 0
# x = np.empty([len(err_prob)])
# for i in tqdm(range(len(err_prob))):
#     x[i] = isHeavy('dc', n_qubit, iamp, istate, err_prob[i], atom_loss, n_trials)[2]
#     print(x[i])

# plt.plot(x,err_prob)
# plt.show()


for i in tqdm(range(trials)):
    init_amps, init_states = create_init_state(n_qubit)
    istate = init_states[n_qubit - 2]
    iamp = init_amps[n_qubit - 2]

    # iamp[:] = 0
    # iamp[-1] = 1.
    # iamp[0:4] = np.array([0.61969722+0.j, 0.64875737+0.j, 0.42085329+0.j, 0.13405875+0.j])
    # iamp[4:] = 0

    left, right = 0, len(err_prob) - 1
    result = None
    #for k in range(len(istate)): print(istate[k],': ', iamp[k])
    #print('\n')

    # for i in range(len(err_prob)):
    #     pr = isHeavy('dc', n_qubit, iamp, istate, err_prob[i], atom_loss, n_trials)
    #     #print('final',err_prob[i],1-(2+err_prob[i]*2+err_prob[i]**2)/5.,pr[2])

    while left <= right:

        mid = left + (right - left) // 2
        #print(err_prob[mid],np.arccos(err_prob[mid]),1 - (2 + 2*err_prob[mid]+err_prob[mid]**2)/5, isHeavy('dc', n_qubit, iamp, istate, err_prob[mid], atom_loss, n_trials)[2])
        if not isHeavy('s', n_qubit, iamp, istate, err_prob[mid], atom_loss, n_trials)[0]:
            result = err_prob[mid]
            left = mid + 1
        else:
            right = mid - 1
    all_mer[i] = result
    print(result)

error = 1-(2+np.array(all_mer)*2+np.array(all_mer)**2)/5.
print(np.mean(error), np.std(error))



In [ ]:
max_qubits = 10
n_trials = min(999,2**max_qubits)
benchmark = 2./3.
atom_loss = 1
q_range = range(3,max_qubits+1)

# sampling cnot error probs
a1 =  np.logspace(-5, -2, 3)
a2 = np.arange(1,15,1)
err_probg = np.outer(a1, a2).flatten()
probg = 1 - np.outer(a1, a2).flatten()
print(len(probg),  np.sqrt((1 + probg**2)/2.), '\n', probg)

s_QV, du_QV, dc_QV, dau_QV, dac_QV = max_qubits+1, max_qubits+1, max_qubits+1, max_qubits+1, max_qubits+1
all_s_QV, all_du_QV, all_dc_QV, all_dau_QV, all_dac_QV = np.full((len(probg)),max_qubits+1), np.full((len(probg)),max_qubits+1), np.full((len(probg)),max_qubits+1), np.full((len(probg)),max_qubits+1), np.full((len(probg)),max_qubits+1)

plotn_cnots_du, plotn_cnots_dc, plotn_cnots_static = [0]*len(probg), [0]*len(probg), [0]*len(probg)

#calculating random initial states
init_amps, init_states = create_init_state(max_qubits)

In [ ]:
for j in tqdm(range(len(probg))):

    print(j)
    atom_loss = 1. #0.9995
    cnot_prob = probg[j]
    s_QV, s_cnots, s_all_qubits_hps = hop_calc('s', max_qubits, init_amps, init_states, cnot_prob, atom_loss, n_trials)
    dc_QV, dc_cnots, dc_all_qubits_hps = hop_calc('dc', max_qubits, init_amps, init_states, cnot_prob, atom_loss, n_trials)
    du_QV, du_cnots, du_all_qubits_hps = hop_calc('du', max_qubits, init_amps, init_states, cnot_prob, atom_loss, n_trials)
    print(s_all_qubits_hps, dc_all_qubits_hps)
    all_s_QV[j], all_dc_QV[j] = s_QV, dc_QV
    all_du_QV[j] = du_QV

    plotn_cnots_static[j], plotn_cnots_dc[j] = s_cnots[s_QV-2], dc_cnots[dc_QV-2]
    plotn_cnots_du[j] = du_cnots[du_QV-2]

    all_s_QV[j], all_dc_QV[j] = s_QV, dc_QV
    all_du_QV[j] = du_QV


indices = np.argsort(-probg)
probg = probg[indices]
all_s_QV = all_s_QV[indices]
all_dc_QV = all_dc_QV[indices]
all_du_QV = all_du_QV[indices]

In [ ]:
fig, ax = plt.subplots(figsize = (9, 6))
alpha = 0.5
# ax.scatter(1. - probg, all_dc_QV, s=60, alpha=alpha, edgecolors="black", color = "blue")
# ax.scatter(1. - probg, all_s_QV, s=60, alpha=alpha, edgecolors="black", color = 'yellow')

ax.plot(1. - probg, all_s_QV, 'o--', alpha=alpha, label = 'static QV')
ax.plot(1. - probg, all_dc_QV, 'o--', alpha=alpha,label = 'dynamic compiled QV')
ax.plot(1. - probg, all_du_QV, 'o--', alpha=alpha, label = 'dynamic not compiled QV')
plt.grid()
plt.xlabel("(1/cnot error)")
plt.ylabel("$\mathbf{log_2V_Q}$")
plt.title("Quantum Volume v/s CNOT gate error")
plt.legend()
# Set logarithmic scale on the x variable
ax.set_xscale("log")

#ADD YOUR PATH HERE CHETANAAA
path = f"D:\\IITBombay\\useqip\\URA\\DIG\\quantum_volume\\plots"
plt.savefig(path + '/qvs_final' + str(max_qubits) + 'prob' + str(len(probg)) + '.png', bbox_inches = 'tight')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (9, 6))
alpha = 0.5
#ax.scatter(1. - probg, plotn_cnots_dynamic, s=60, alpha=alpha, edgecolors="black", color = "blue")
#ax.scatter(1. - probg, plotn_cnots_static, s=60, alpha=alpha, edgecolors="black", color = 'yellow')
ax.plot(1. - probg, plotn_cnots_static, 'o--', alpha=alpha,label = 'static')
ax.plot(1. - probg, plotn_cnots_dc, 'o--', alpha=alpha, label = 'dynamic compiled')
ax.plot(1. - probg, plotn_cnots_du, 'o--', alpha=alpha, label = 'dynamic not compiled')
plt.xlabel("cnot error")
plt.ylabel("Number of gates")
plt.title("Number of gates v/s CNOT gate error")

plt.legend()
# Set logarithmic scale on the x variable
ax.set_xscale("log")
#ax.set_yscale("log")

path = f"D:\\IITBombay\\useqip\\URA\\DIG\\quantum_volume\\plots"
plt.savefig(path + '/ncnots_final' + str(max_qubits) + 'prob' + str(len(probg)) + '.png', bbox_inches = 'tight')
plt.show()